# How (not) to talk with Polish Police

In [ ]:
import pandas as pd
from transformers import pipeline
# Load the Excel file
file_path = '/content/Summer School project - cases (english version) - updated + GPT.xlsx'
df = pd.read_excel(file_path)
# Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')
# Function to get sentiment score
def get_sentiment_score(text):
    result = sentiment_pipeline(text)
    return result[0]['label'], result[0]['score']
# Apply sentiment analysis on the "Words used" column
df['Sentiment'], df['Score'] = zip(*df['Words used'].apply(get_sentiment_score))
# Display the results
print(df[['Words used', 'Sentiment', 'Score']])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that aut

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

                                           Words used Sentiment     Score
0   A piece of shit. A thief. An idiot. A cheater....  NEGATIVE  0.999746
1   You are a failure in life. You are uneducated....  NEGATIVE  0.999469
2   Jump out of your uniform so I can fucking tap ...  NEGATIVE  0.950563
3                   You are a criminal, a degenerate.  NEGATIVE  0.996724
4   Flapper. A man whose face resembles a person f...  NEGATIVE  0.997221
5   The police should shove these tickets up their...  NEGATIVE  0.999482
6   Comedian. You are dilettante and unaware of le...  NEGATIVE  0.999703
7   I have the impression, or rather certainty, th...  POSITIVE  0.683263
8   The laurel of awarding the idiot medal to an i...  POSITIVE  0.686951
9          You pricks, whores, fucking dogs, fuckers.  NEGATIVE  0.997408
10                              You ladies are crazy.  POSITIVE  0.996169
11  Idiot. You can't read with understanding. The ...  POSITIVE  0.812648
12            You boor, loser, don't c

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load data from Excel file
file_path = '/content/Summer School project - cases (english version) - updated + GPT.xlsx'
df = pd.read_excel(file_path)

# Verify column names
print(df.columns)

# Extract the relevant columns
texts = df['Words used']
labels = df['Judgment']

# Ensure at least one sample from each class is in both training and testing sets
stratify = labels if labels.nunique() > 1 else None
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.5, random_state=42, stratify=stratify)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Function to predict based on user input
def predict_insult():
    while True:
        user_input = input("Enter the sentence (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        if user_input:
            user_input_vec = vectorizer.transform([user_input])
            prediction = model.predict(user_input_vec)
            result = "Guilty of insulting a public official" if prediction[0] == 1 else "Not guilty of insulting a public official"
            print("Prediction:", result)

# Run the prediction function
if __name__ == "__main__":
    predict_insult()

Index(['Court', 'Date', 'Case number', 'Words used', 'Context', 'Judgment'], dtype='object')
Accuracy: 0.75
              precision    recall  f1-score   support

           0       0.68      0.93      0.79        14
           1       0.89      0.57      0.70        14

    accuracy                           0.75        28
   macro avg       0.79      0.75      0.74        28
weighted avg       0.79      0.75      0.74        28

Prediction: Not guilty of insulting a public official
Prediction: Guilty of insulting a public official
